In [1]:
# Carregando algumas bibliotecas importantes
import numpy as np
import pandas as pd
from datetime import datetime

In [2]:
# Importando os arquivos
jogos = pd.read_csv('../data/raw/matches_FLA2025.csv', date_format={'kick-off': '%Y-%m-%d %H:%M:%S'})
gols = pd.read_csv('../data/raw/goal_scorers_2025.csv')

In [3]:
jogos['kick-off'] = pd.to_datetime(jogos['kick-off'], errors='coerce')
jogos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   id_match    54 non-null     object        
 1   home_team   54 non-null     object        
 2   away_team   54 non-null     object        
 3   home_score  54 non-null     int64         
 4   away_score  54 non-null     int64         
 5   tournament  54 non-null     object        
 6   kick-off    54 non-null     datetime64[ns]
 7   stadium     54 non-null     object        
 8   attendance  54 non-null     int64         
 9   notes       9 non-null      object        
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 4.3+ KB


In [4]:
# Mostrando as 5 primeiras linhas do dataframe jogos
jogos.head()

,id_match,home_team,away_team,home_score,away_score,tournament,kick-off,stadium,attendance,notes
0,CA1,Flamengo,Boa Vista-RJ,1,2,Carioca - Round 1,2025-01-12 16:00:00,Estádio Batistão-SE,3793,time sub-20
1,CA2,Madureira-RJ,Flamengo,1,1,Carioca - Round 2,2025-01-16 18:30:00,Estádio Governador Ernani Sátyro-PB,3614,time sub-20
2,AM1,São Paulo,Flamengo,0,0,International Friendly Game,2025-01-19 17:00:00,Chase Stadium-USA,21550,amistoso
3,CA3,Flamengo,Nova Iguaçu-RJ,1,2,Carioca - Round 3,2025-01-19 18:30:00,Estádio Governador Ernani Sátyro-PB,3614,time sub-20
4,CA4,Bangu,Flamengo,0,5,Carioca - Round 4,2025-01-22 19:00:00,Estádio Governador João Castelo-MA,29820,time sub-20


In [5]:
# Mostrando as 5 ultimas linhas do dataframe jogos
jogos.tail()

,id_match,home_team,away_team,home_score,away_score,tournament,kick-off,stadium,attendance,notes
49,LIB7,Flamengo,Internacional,1,0,Copa Libertadores - Round of 16 – First Leg,2025-08-13 21:30:00,Maracanã-RJ,68483,NaN
50,BR19,Internacional,Flamengo,1,3,Campeonato Brasileiro - Round 19,2025-08-17 18:30:00,Beira Rio-RS,22241,NaN
51,LIB8,Internacional,Flamengo,0,2,Copa Libertadores - Round of 16 - Second Leg,2025-08-20 21:30:00,Beira Rio-RS,43581,Classificado para as quartas de finais
52,BR19,Flamengo,Vitória,8,0,Campeonato Brasileiro - Round 20,2025-08-25 21:00:00,Maracanã-RJ,60642,NaN
53,BR20,Flamengo,Grêmio,1,1,Campeonato Brasileiro - Round 21,2025-08-31 16:00:00,Maracanã-RJ,69447,NaN


In [6]:
# Mostrando as 5 primeiras linhas do dataframe jogos
gols.head()

,id_match,gols,assist
0,CA1,Carlinhos,NaN
1,CA2,Thiaguinho,NaN
2,CA3,Wallace Yan,Guilherme
3,CA4,Felipe Teresa,Wallace Yan
4,CA4,Wallace Yan,Felipe Teresa


In [7]:
# Mostrando as 5 ultimas linhas do dataframe jogos
gols.tail()

,id_match,gols,assist
96,BR20,Samuel Lino,Varela
97,BR20,Luiz Araújo,Pedro
98,BR20,Pedro,Samuel Lino
99,BR20,Bruno Henrique,Pênalti
100,BR21,Arrascaeta,Pedro


### Criando a coluna torneio

In [9]:
# Split o nome do torneio e a rodada
lista = [i.split(' -', 1)[0] for i in jogos["tournament"].tolist()]

# Transformando em uma nova coluna chamada torneio
jogos['torneio'] = np.array(lista)

### Criando a coluna local (casa/fora)

In [11]:
def obter_locale(row):
    if row['torneio'] == 'Supercopa do Brasil' or row['torneio'] == 'Club World Championship':
        return 'Neutro'
    elif row['home_team'] == 'Flamengo':
        return 'Casa'
    else:
        return 'Fora'

    
# Aplicar a função para criar a nova coluna
jogos['locale'] = jogos.apply(obter_locale, axis=1)

### Criando a coluna resultado

In [13]:
# Função para obter resultado
def obter_resultado(row):
    if row['home_team'] == 'Flamengo':
        gols_marcados = row['home_score']
        gols_sofridos = row['away_score']
        
        if gols_marcados < gols_sofridos:
            return 'Derrota'
        elif gols_marcados > gols_sofridos:
            return 'Vitória'
        else:
            return 'Empate'
    if row['away_team'] == 'Flamengo':
        gols_marcados = row['away_score']
        gols_sofridos = row['home_score']

        if gols_marcados < gols_sofridos:
            return 'Derrota'
        elif gols_marcados > gols_sofridos:
            return 'Vitória'
        else:
            return 'Empate'

# Aplicar a função para criar a nova coluna
jogos['resultado'] = jogos.apply(obter_resultado, axis=1)

### Criando mais algumas colunas 

In [15]:
def gerar_estatisticas_gols(row):
    if row['home_team'] == 'Flamengo':
        gols_feitos_casa = row['home_score']
        gols_sofridos_casa = row['away_score']
        gols_feitos_fora = 0
        gols_sofridos_fora = 0
        marcou_gol = 'Sim' if row['home_score'] > 0 else 'Não'
        sofreu_gol = 'Sim' if row['away_score'] > 0 else 'Não'
    elif row['away_team'] == 'Flamengo':
        gols_feitos_casa = 0
        gols_sofridos_casa = 0
        gols_feitos_fora = row['away_score']
        gols_sofridos_fora = row['home_score']
        marcou_gol = 'Sim' if row['away_score'] > 0 else 'Não'
        sofreu_gol = 'Sim' if row['home_score'] > 0 else 'Não'
    else:
        gols_feitos_casa = 0
        gols_sofridos_casa = 0
        gols_feitos_fora = 0
        gols_sofridos_fora = 0
        marcou_gol = 'Não'
        sofreu_gol = 'Não'

    return pd.Series([gols_feitos_casa, gols_sofridos_casa, gols_feitos_fora, gols_sofridos_fora, marcou_gol, sofreu_gol])

jogos[['gols_casa', 'gols_sofridos_casa', 'gols_fora', 'gols_sofridos_fora', 'marcou_gol', 'sofreu_gol']] = jogos.apply(gerar_estatisticas_gols, axis=1)        

jogos['jogos_cumulativos'] = range(1, len(jogos) + 1)

#### Salvando os dataframes 

In [17]:
jogos['year'] = jogos['kick-off'].map(lambda x: datetime.strftime(x,"%Y"))
jogos['month'] = jogos['kick-off'].map(lambda x: datetime.strftime(x,"%b"))
jogos['weekday'] = jogos['kick-off'].map(lambda x: datetime.strftime(x,"%a"))

In [18]:
jogos.to_csv('../data/processed/matches_FLA2025_processed.csv', index=False)
gols.to_csv('../data/processed/goal_scorers_2025_processed.csv', index=False)

In [19]:
jogos

,id_match,home_team,away_team,home_score,away_score,tournament,kick-off,stadium,attendance,notes,...,gols_casa,gols_sofridos_casa,gols_fora,gols_sofridos_fora,marcou_gol,sofreu_gol,jogos_cumulativos,year,month,weekday
0,CA1,Flamengo,Boa Vista-RJ,1,2,Carioca - Round 1,2025-01-12 16:00:00,Estádio Batistão-SE,3793,time sub-20,...,1,2,0,0,Sim,Sim,1,2025,Jan,Sun
1,CA2,Madureira-RJ,Flamengo,1,1,Carioca - Round 2,2025-01-16 18:30:00,Estádio Governador Ernani Sátyro-PB,3614,time sub-20,...,0,0,1,1,Sim,Sim,2,2025,Jan,Thu
2,AM1,São Paulo,Flamengo,0,0,International Friendly Game,2025-01-19 17:00:00,Chase Stadium-USA,21550,amistoso,...,0,0,0,0,Não,Não,3,2025,Jan,Sun
3,CA3,Flamengo,Nova Iguaçu-RJ,1,2,Carioca - Round 3,2025-01-19 18:30:00,Estádio Governador Ernani Sátyro-PB,3614,time sub-20,...,1,2,0,0,Sim,Sim,4,2025,Jan,Sun
4,CA4,Bangu,Flamengo,0,5,Carioca - Round 4,2025-01-22 19:00:00,Estádio Governador João Castelo-MA,29820,time sub-20,...,0,0,5,0,Sim,Não,5,2025,Jan,Wed
5,CA5,Volta Redonda,Flamengo,0,2,Carioca - Round 5,2025-01-25 16:30:00,Estádio Nacional Mané Garrincha-DF,23872,NaN,...,0,0,2,0,Sim,Não,6,2025,Jan,Sat
6,CA6,Flamengo,Sampaio Correia-RJ,2,0,Carioca - Round 6,2025-01-30 21:30:00,Maracanã-RJ,18423,NaN,...,2,0,0,0,Sim,Não,7,2025,Jan,Thu
7,SC1,Botafogo,Flamengo,1,3,Supercopa do Brasil,2025-02-02 16:00:00,Estádio Olímpico do Pará-PA,44952,NaN,...,0,0,3,1,Sim,Sim,8,2025,Feb,Sun
8,CA8,Portuguesa-RJ,Flamengo,0,5,Carioca - Round 7,2025-02-05 19:00:00,Estádio Parque do Sabiá-MG,16396,NaN,...,0,0,5,0,Sim,Não,9,2025,Feb,Wed
9,CA9,Fluminense,Flamengo,0,0,Carioca - Round 8,2025-02-08 16:30:00,Maracanã-RJ,34617,NaN,...,0,0,0,0,Não,Não,10,2025,Feb,Sat
